In [1]:
import dart_fss as dart
import pandas as pd
import os


api_key='1ed4a4873cb9557629a64dc0c163cb2bb438f3a4'
dart.set_api_key(api_key=api_key)

#  삼우 code
corp_code_list = ['00237710', '00266961', '00258801', '00130897', '00254045', '00386937', '00201692', ]

# 데이터 프레임 생성
df_income = pd.DataFrame()

for corp_code in corp_code_list:
    # 모든 상장된 기업 리스트 불러오기
    try:
        corp_list = dart.get_corp_list()

        # 삼성전자
        samwoo = corp_list.find_by_corp_code(corp_code=corp_code)

        # 2012년 01월 01일 부터 연결재무제표 검색
        # fs = samsung.extract_fs(bgn_de='20120101') 와 동일
        fs = dart.fs.extract(corp_code=corp_code, bgn_de='20170101',separate=True)


        # ,separate=True
        # 연결재무상태표
        df_fs = fs['bs'] # 또는 df = fs[0] 또는 df = fs.show('bs')
        # 연결재무상태표 추출에 사용된 Label 정보
        labels_fs = fs.labels['bs']

        # 연결손익계산서
        df_is = fs['is'] # 또는 df = fs[1] 또는 df = fs.show('is')
        # 연결손익계산서 추출에 사용된 Label 정보
        labels_is = fs.labels['is']

        # 연결포괄손익계산서
        df_ci = fs['cis'] # 또는 df = fs[2] 또는 df = fs.show('cis')
        # 연결포괄손익계산서 추출에 사용된 Label 정보
        labels_ci = fs.labels['cis']

        # 현금흐름표
        df_cf = fs['cf'] # 또는 df = fs[3] 또는 df = fs.show('cf')
        # 현금흐름표 추출에 사용된 Label 정보
        labels_cf = fs.labels['cf']

        # 재무제표 일괄저장 (default: 실행폴더/fsdata/{corp_code}_{report_tp}.xlsx)
        # fs.save()

        # 재무제표 일괄저장
        filename = str(corp_code) + '_rev01.xlsx'

        # fsdata 경로 지정 
        path = os.path.join(os.path.curdir, "fsdata")

        # fsdata 디렉토리가 없으면 만들고 파일 경로 지정
        if os.path.isdir(path) != True:
            os.mkdir(path)
            file_path = os.path.join(path, filename)
        else:
            file_path = os.path.join(path, filename)
        fs.save(filename,path)




        # data_is / data_cis sheet 찾기
        sheet_name = [i for i in (pd.ExcelFile(file_path).sheet_names) if "data" and "is" in str(i).lower() and "label" not in str(i).lower()][0]

        df = pd.read_excel(file_path, sheet_name=sheet_name)

        # "label_ko"가 있는 행을 인덱스로 지정 -> 영업이익을 인덱스로 활용하기 위해
        label_ko_idx = [idx for idx, val in enumerate(df.iloc[0, :].isin(["label_ko"])) if val==True][0]
        df.set_index(df.columns[label_ko_idx], inplace=True)

        # 영업이익 추출
        income_idx = [i for i in list(df.index) if "영업이익" in str(i)][0]
        income_col = ['20200101-20201231', '20190101-20191231','20180101-20181231', 
                  '20170101-20171231', '20160101-20161231','20150101-20151231']
        df = df.loc[[income_idx], income_col]

        # 인덱스를 기업 코드로 변경
        df.rename(index={income_idx: corp_code}, inplace=True)

        # df_income 에 concat
        if len(df_income) == 0:
            for col in df.columns:
                df_income[col] = df[col]
        else:
            df_income = pd.concat([df_income, df])
            
    except:
        print(corp_code, "의 기업 정보가 없습니다.")
        pass

Output()

Output()

Output()


00130897 의 기업 정보가 없습니다.



00201692 의 기업 정보가 없습니다.


In [2]:
df_income

,20200101-20201231,20190101-20191231,20180101-20181231,20170101-20171231,20160101-20161231,20150101-20151231
00237710,5893759098,1205494027,2788338039,2560462984,4149382043,2392679271
00266961,1440813731880,1393811994426,1263548953129,1178237472725,968350907579,826491297771
00258801,300064492177,196063924523,122366055178,97550822849,70994504761,120590601375
00254045,1765336,2471553,2415691,1790558,1270501,1090372
00386937,3630433000000,3432228000000,3116896000000,2757771000000,1432923000000,1903599000000


In [7]:
df_income.to_excel("income_report.xlsx")

In [10]:
df = pd.read_excel("income_report.xlsx",index_col="Unnamed: 0")
df

,20200101-20201231,20190101-20191231,20180101-20181231,20170101-20171231,20160101-20161231,20150101-20151231
237710,5893759098,1205494027,2788338039,2560462984,4149382043,2392679271
266961,1440813731880,1393811994426,1263548953129,1178237472725,968350907579,826491297771
258801,300064492177,196063924523,122366055178,97550822849,70994504761,120590601375
254045,1765336,2471553,2415691,1790558,1270501,1090372
386937,3630433000000,3432228000000,3116896000000,2757771000000,1432923000000,1903599000000
